<a href="https://colab.research.google.com/github/narayankundu/NK-Python-Codes/blob/main/electron_ecollision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#!/usr/bin/env python3
"""
Strong-field recollision probe of molecular chirality
Scientific implementation with proper quantum models

Reference:
- Theory: Phys. Rev. A 94, 033401 (2016)
- Chirality: Nat. Phys. 13, 276 (2017)
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import physical_constants, c, m_e, e, hbar
from scipy.integrate import odeint, solve_ivp
from scipy.special import gamma, airy
from scipy.optimize import brentq
import numpy.polynomial.hermite as herm
from rdkit import Chem
from rdkit.Chem import AllChem
from pyscf import gto, scf, dft, lib
from pyscf.hessian import thermo
from pyscf.data.nist import BOHR

# Atomic units
au_energy = physical_constants['atomic unit of energy'][0]
au_time = physical_constants['atomic unit of time'][0]
au_field = physical_constants['atomic unit of electric field'][0]
au_velocity = physical_constants['atomic unit of velocity'][0]

########################
# Quantum Mechanical Models
########################

class StrongFieldModels:
    """Implementation of strong-field physics models"""

    @staticmethod
    def ADK_rate(Ip, F, l=0, m=0, Z=1):
        """
        Ammosov-Delone-Krainov tunnel ionization rate
        Phys. Rev. A 64, 013416 (2001)
        """
        E_h = 1.0  # Hartree energy in au
        n_eff = Z / np.sqrt(2 * Ip)

        # Angular momentum dependent terms
        if l == 0:
            C_lm = 1.0
        elif l == 1 and m == 0:
            C_lm = 2.0
        else:
            C_lm = 1.0  # Simplified for other states

        F_au = F / au_field

        prefactor = C_lm**2 * (2** (2*n_eff)) / (n_eff * gamma(2*n_eff))
        exponent = -(2 * (2*Ip)**1.5) / (3 * F_au)

        rate = prefactor * ( (2*Ip)**1.5 / F_au )**(2*n_eff - 1) * np.exp(exponent)
        return rate * (E_h / hbar)  # Convert to 1/s

    @staticmethod
    def tunnel_exit_point(Ip, F, theta=0):
        """
        Calculate tunnel exit point including molecular orientation
        Phys. Rev. A 85, 023405 (2012)
        """
        F_au = F / au_field
        # Simple model: exit where potential barrier equals -Ip
        x_exit = -Ip / F_au  # Atomic units
        return x_exit

    @staticmethod
    def initial_momentum_distribution(Ip, F, transverse=True):
        """
        Initial transverse momentum distribution
        Phys. Rev. Lett. 71, 1994 (1993)
        """
        F_au = F / au_field
        if transverse:
            # Perpendicular momentum distribution
            p_perp_max = np.sqrt(F_au / np.sqrt(2*Ip))
            return p_perp_max
        else:
            # Parallel momentum (approximately zero at tunnel exit)
            return 0.0

    @staticmethod
    def molecular_potential(r, coords, charges, method='coulomb'):
        """
        Molecular ionic potential with proper multi-center structure
        """
        V = 0.0
        r = np.array(r)

        for i, coord in enumerate(coords):
            R_vec = r - coord
            R = np.linalg.norm(R_vec)

            if method == 'soft_coulomb':
                # Softened Coulomb potential
                a = 1.0  # Softening parameter
                V += -charges[i] / np.sqrt(R**2 + a**2)
            elif method == 'coulomb':
                # Pure Coulomb potential
                V += -charges[i] / R
            elif method == 'model_potential':
                # Model potential with exponential decay
                alpha = 1.0  # Screening parameter
                V += -charges[i] * np.exp(-alpha * R) / R

        return V

    @staticmethod
    def molecular_force(r, coords, charges):
        """Force from molecular ionic potential"""
        F = np.zeros(3)
        r = np.array(r)
        epsilon = 1e-8  # Small regularization

        for i, coord in enumerate(coords):
            R_vec = r - coord
            R = np.linalg.norm(R_vec) + epsilon
            # Coulomb force
            F += -charges[i] * R_vec / R**3

        return F

########################
# Chiral-Sensitive Interactions
########################

class ChiralInteractions:
    """Chiral-sensitive interaction terms"""

    @staticmethod
    def magnetic_dipole_interaction(v, B, mu_B=1/2):
        """
        Magnetic dipole interaction for chiral sensitivity
        Includes electron magnetic moment
        """
        # Bohr magneton in atomic units
        mu_B_au = 1/2  # Approximately 1/2 in au
        return -mu_B_au * np.dot(v, B)  # Simplified interaction

    @staticmethod
    def effective_chiral_potential(r, v, coords, chiral_parameter=1e-3):
        """
        Effective chiral potential modeling handedness
        This is a phenomenological model for demonstration
        """
        # Simple model: velocity-dependent potential that breaks mirror symmetry
        chiral_strength = chiral_parameter
        # Create a chiral field based on molecular structure
        center_of_mass = np.mean(coords, axis=0)
        R = r - center_of_mass
        R_norm = np.linalg.norm(R) + 1e-8

        # Pseudoscalar term that changes sign with chirality
        chiral_term = chiral_strength * np.dot(np.cross(R, v), [0, 0, 1]) / R_norm**2
        return chiral_term

########################
# Laser Field with Envelope
########################

class LaserField:
    """Realistic laser field with proper envelope"""

    def __init__(self, I0=2e14, wavelength=800e-9, pulse_duration=30e-15,
                 polarization='circular', chirality='right'):
        self.I0 = I0  # W/cm²
        self.wavelength = wavelength
        self.omega = 2 * np.pi * c / wavelength  # Angular frequency
        self.omega_au = self.omega * au_time  # Convert to atomic units

        # Pulse parameters
        self.tau = pulse_duration  # Pulse duration
        self.tau_au = pulse_duration / au_time

        self.polarization = polarization
        self.chirality = chirality

        # Calculate electric field amplitude
        self.E0 = np.sqrt(I0 * 1e4 / (c * 8.854e-12))  # V/m
        self.E0_au = self.E0 / au_field  # Atomic units

    def envelope(self, t, t0=0):
        """Gaussian envelope"""
        return np.exp(-2 * np.log(2) * ((t - t0) / self.tau_au)**2)

    def electric_field(self, t):
        """Electric field with envelope"""
        envelope = self.envelope(t)

        if self.polarization == 'linear':
            Ex = self.E0_au * envelope * np.cos(self.omega_au * t)
            Ey = 0.0
            Ez = 0.0
        elif self.polarization == 'circular':
            phase = np.pi/2 if self.chirality == 'right' else -np.pi/2
            Ex = self.E0_au * envelope * np.cos(self.omega_au * t)
            Ey = self.E0_au * envelope * np.cos(self.omega_au * t + phase)
            Ez = 0.0
        elif self.polarization == 'elliptical':
            # Elliptical polarization for chiral studies
            phase = np.pi/2 if self.chirality == 'right' else -np.pi/2
            Ex = self.E0_au * envelope * np.cos(self.omega_au * t)
            Ey = 0.7 * self.E0_au * envelope * np.cos(self.omega_au * t + phase)
            Ez = 0.0
        else:
            raise ValueError("Unsupported polarization")

        return np.array([Ex, Ey, Ez])

    def vector_potential(self, t):
        """Vector potential A(t) = -∫E(t)dt"""
        if self.polarization == 'linear':
            Ax = -(self.E0_au / self.omega_au) * self.envelope(t) * np.sin(self.omega_au * t)
            Ay = 0.0
        elif self.polarization == 'circular':
            phase = np.pi/2 if self.chirality == 'right' else -np.pi/2
            Ax = -(self.E0_au / self.omega_au) * self.envelope(t) * np.sin(self.omega_au * t)
            Ay = -(self.E0_au / self.omega_au) * self.envelope(t) * np.sin(self.omega_au * t + phase)
        else:
            Ax, Ay = 0.0, 0.0

        return np.array([Ax, Ay, 0.0])

    def magnetic_field(self, t):
        """Magnetic field for circular polarization"""
        if self.polarization == 'circular':
            # B = (1/c) * k × E for plane wave
            k_mag = self.omega_au / c  # Wavevector magnitude in au
            E = self.electric_field(t)
            B = np.array([-E[1], E[0], 0.0]) * k_mag
            return B
        else:
            return np.zeros(3)

########################
# Molecular Setup with Chirality
########################

def build_chiral_molecule(molecule="fenchone", enantiomer="R"):
    """
    Build chiral molecule geometry with proper stereochemistry
    """
    if molecule.lower() == "fenchone":
        if enantiomer.upper() == "R":
            smiles = "C[C@]12CCC(C1(C)C(=O)C2)O"  # R-fenchone
        else:
            smiles = "C[C@@]12CCC(C1(C)C(=O)C2)O"  # S-fenchone
    elif molecule.lower() == "methylene_oxirane":
        if enantiomer.upper() == "R":
            smiles = "C[C@H]1CO1"  # R-methylene oxirane
        else:
            smiles = "C[C@@H]1CO1"  # S-methylene oxirane
    else:
        raise ValueError("Molecule not recognized")

    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)

    # Embed with stereochemistry preservation
    ps = AllChem.ETKDGv3()
    ps.randomSeed = 42
    AllChem.EmbedMolecule(mol, ps)
    AllChem.MMFFOptimizeMolecule(mol)

    conf = mol.GetConformer()
    coords = np.array([list(conf.GetAtomPosition(i)) for i in range(mol.GetNumAtoms())])
    symbols = [atom.GetSymbol() for atom in mol.GetAtoms()]

    # Convert to atomic units (Bohr)
    coords_au = coords * 1e-10 / BOHR

    return symbols, coords_au, mol

########################
# Electronic Structure Calculation
########################

def calculate_molecular_properties(symbols, coords):
    """
    Calculate molecular properties using PySCF
    """
    atom_str = ""
    for s, (x, y, z) in zip(symbols, coords):
        atom_str += f"{s} {x} {y} {z}; "

    mol = gto.Mole()
    mol.atom = atom_str
    mol.basis = '6-31g*'  # Better basis for chirality
    mol.verbose = 0
    mol.build()

    # Run calculation
    mf = scf.RHF(mol).run()

    # Get molecular properties
    Ip = -mf.mo_energy[mf.mo_occ > 0][-1]  # Ionization potential
    dipole = mf.dip_moment(unit='Debye')

    # Calculate molecular orbitals for visualization
    mo_coeff = mf.mo_coeff
    mo_energy = mf.mo_energy

    return {
        'ionization_potential': Ip,
        'dipole_moment': dipole,
        'mo_coeff': mo_coeff,
        'mo_energy': mo_energy,
        'mol': mol,
        'mf': mf
    }

########################
# Trajectory Simulation with Quantum Initial Conditions
########################

class ElectronTrajectory:
    """Quantum-classical electron trajectory with proper initial conditions"""

    def __init__(self, mol_properties, laser, chiral_interactions=True):
        self.mol_properties = mol_properties
        self.laser = laser
        self.chiral_interactions = chiral_interactions

        self.coords = mol_properties['mol'].atom_coords()
        self.charges = mol_properties['mol'].atom_charges()
        self.Ip = mol_properties['ionization_potential']

    def sample_tunnel_ionization(self, n_trajectories=1000):
        """
        Sample tunnel ionization times and initial conditions using ADK theory
        """
        # Time sampling within laser pulse
        t_start = -2 * self.laser.tau_au
        t_end = 2 * self.laser.tau_au
        times = np.linspace(t_start, t_end, 1000)

        # Calculate ionization rates
        rates = []
        for t in times:
            E_field = np.linalg.norm(self.laser.electric_field(t))
            if E_field > 0:
                rate = StrongFieldModels.ADK_rate(self.Ip, E_field * au_field)
                rates.append(rate)
            else:
                rates.append(0.0)

        rates = np.array(rates)

        # Sample ionization times
        total_rate = np.trapz(rates, times)
        probabilities = rates / total_rate if total_rate > 0 else rates

        tunnel_times = []
        initial_conditions = []

        for _ in range(n_trajectories):
            # Sample ionization time
            t0 = np.random.choice(times, p=probabilities/np.sum(probabilities))

            # Get field at ionization time
            E_t0 = self.laser.electric_field(t0)
            E_mag = np.linalg.norm(E_t0)

            if E_mag > 0:
                # Tunnel exit point
                r0 = StrongFieldModels.tunnel_exit_point(self.Ip, E_mag * au_field)
                direction = E_t0 / E_mag
                exit_point = r0 * direction

                # Initial momentum (transverse distribution)
                p_perp_max = StrongFieldModels.initial_momentum_distribution(self.Ip, E_mag * au_field)
                p_perp = np.random.normal(0, p_perp_max/2, 2)  # Two transverse directions
                p_parallel = 0.0  # Zero initial longitudinal momentum

                # Combine momenta
                p0 = np.zeros(3)
                p0[:2] = p_perp

                initial_conditions.append({
                    't0': t0,
                    'r0': exit_point,
                    'p0': p0,
                    'ionization_rate': StrongFieldModels.ADK_rate(self.Ip, E_mag * au_field)
                })

        return initial_conditions

    def equations_of_motion(self, t, y):
        """
        Equations of motion with molecular potential and laser field
        """
        r, p = y[:3], y[3:]

        # Laser electric field
        E_laser = self.laser.electric_field(t)

        # Molecular force
        F_mol = StrongFieldModels.molecular_force(r, self.coords, self.charges)

        # Total force
        dpdt = -F_mol + E_laser

        # Velocity (p = v in atomic units for electron)
        drdt = p

        # Chiral interactions if enabled
        if self.chiral_interactions:
            B_laser = self.laser.magnetic_field(t)
            chiral_force = ChiralInteractions.magnetic_dipole_interaction(p, B_laser)
            dpdt[2] += chiral_force  # Add to z-component

            # Effective chiral potential
            chiral_potential = ChiralInteractions.effective_chiral_potential(r, p, self.coords)
            dpdt += -chiral_potential * p  # Velocity-dependent force

        return np.concatenate([drdt, dpdt])

    def propagate_trajectory(self, initial_condition, t_max=400):
        """
        Propagate single electron trajectory
        """
        t0 = initial_condition['t0']
        y0 = np.concatenate([initial_condition['r0'], initial_condition['p0']])

        t_span = (t0, t0 + t_max)
        t_eval = np.linspace(t0, t0 + t_max, 2000)

        try:
            sol = solve_ivp(self.equations_of_motion, t_span, y0,
                           t_eval=t_eval, method='RK45', rtol=1e-8, atol=1e-11)

            if sol.success:
                return sol.t, sol.y
            else:
                return None, None
        except:
            return None, None

    def detect_recollision(self, t, trajectory, threshold=2.0):
        """
        Detect recollision events with molecular core
        """
        r = trajectory[:3, :].T  # Position history

        min_distance = float('inf')
        recollision_time = None
        recollision_energy = None

        for i, pos in enumerate(r):
            # Distance to all atoms
            distances = [np.linalg.norm(pos - atom_coord) for atom_coord in self.coords]
            min_dist = min(distances)

            if min_dist < threshold and min_dist < min_distance:
                min_distance = min_dist
                recollision_time = t[i]
                # Kinetic energy at recollision
                v = trajectory[3:, i]  # Velocity
                recollision_energy = 0.5 * np.dot(v, v)

        return recollision_time, recollision_energy, min_distance

########################
# Analysis and Chirality Metrics
########################

class ChiralityAnalysis:
    """Analysis of chiral-sensitive signals"""

    @staticmethod
    def circular_dichroism_signal(recollision_energy_R, recollision_energy_S):
        """
        Calculate circular dichroism in recollision energy
        CD = (I_R - I_S) / (0.5*(I_R + I_S))
        """
        if len(recollision_energy_R) == 0 or len(recollision_energy_S) == 0:
            return 0.0

        mean_R = np.mean(recollision_energy_R)
        mean_S = np.mean(recollision_energy_S)

        if mean_R + mean_S == 0:
            return 0.0

        CD = 2 * (mean_R - mean_S) / (mean_R + mean_S)
        return CD

    @staticmethod
    def asymmetry_parameter(trajectories_R, trajectories_S):
        """
        Calculate trajectory asymmetry parameter
        """
        if len(trajectories_R) == 0 or len(trajectories_S) == 0:
            return 0.0

        # Analyze final momentum distributions
        final_momenta_R = [traj[3:, -1] for traj in trajectories_R if traj is not None]
        final_momenta_S = [traj[3:, -1] for traj in trajectories_S if traj is not None]

        if len(final_momenta_R) == 0 or len(final_momenta_S) == 0:
            return 0.0

        final_momenta_R = np.array(final_momenta_R)
        final_momenta_S = np.array(final_momenta_S)

        # Calculate asymmetry in momentum distribution
        asymmetry = np.mean(final_momenta_R[:, 2]) - np.mean(final_momenta_S[:, 2])
        return asymmetry

########################
# Main Simulation
########################

def run_chiral_simulation():
    """Complete chiral strong-field simulation"""

    print("=== Strong-Field Chiral Recollision Simulation ===")
    print("Initializing molecular systems...")

    # Build both enantiomers
    symbols_R, coords_R, mol_R = build_chiral_molecule("fenchone", "R")
    symbols_S, coords_S, mol_S = build_chiral_molecule("fenchone", "S")

    print(f"R-enantiomer: {len(symbols_R)} atoms")
    print(f"S-enantiomer: {len(symbols_S)} atoms")

    # Calculate molecular properties
    print("Calculating electronic structure...")
    props_R = calculate_molecular_properties(symbols_R, coords_R)
    props_S = calculate_molecular_properties(symbols_S, coords_S)

    print(f"Ionization potential R: {props_R['ionization_potential']:.3f} Ha")
    print(f"Ionization potential S: {props_S['ionization_potential']:.3f} Ha")

    # Initialize laser
    laser_lcp = LaserField(I0=2e14, wavelength=800e-9,
                          polarization='circular', chirality='left')
    laser_rcp = LaserField(I0=2e14, wavelength=800e-9,
                          polarization='circular', chirality='right')

    print(f"Laser intensity: {laser_lcp.I0:.2e} W/cm²")
    print(f"Laser wavelength: {laser_lcp.wavelength*1e9:.1f} nm")

    # Initialize trajectory simulators
    traj_sim_R = ElectronTrajectory(props_R, laser_rcp, chiral_interactions=True)
    traj_sim_S = ElectronTrajectory(props_S, laser_rcp, chiral_interactions=True)

    # Sample ionization events
    print("Sampling tunnel ionization events...")
    initial_conditions_R = traj_sim_R.sample_tunnel_ionization(n_trajectories=500)
    initial_conditions_S = traj_sim_S.sample_tunnel_ionization(n_trajectories=500)

    print(f"Sampled {len(initial_conditions_R)} ionization events for R-enantiomer")
    print(f"Sampled {len(initial_conditions_S)} ionization events for S-enantiomer")

    # Propagate trajectories
    print("Propagating electron trajectories...")

    results_R = {'trajectories': [], 'recollision_energies': []}
    results_S = {'trajectories': [], 'recollision_energies': []}

    for i, (ic_R, ic_S) in enumerate(zip(initial_conditions_R, initial_conditions_S)):
        if i % 50 == 0:
            print(f"Progress: {i}/{len(initial_conditions_R)}")

        # Propagate R-enantiomer
        t_R, traj_R = traj_sim_R.propagate_trajectory(ic_R)
        if traj_R is not None:
            results_R['trajectories'].append((t_R, traj_R))
            # Check for recollision
            t_rec, E_rec, dist = traj_sim_R.detect_recollision(t_R, traj_R)
            if E_rec is not None:
                results_R['recollision_energies'].append(E_rec)

        # Propagate S-enantiomer
        t_S, traj_S = traj_sim_S.propagate_trajectory(ic_S)
        if traj_S is not None:
            results_S['trajectories'].append((t_S, traj_S))
            # Check for recollision
            t_rec, E_rec, dist = traj_sim_S.detect_recollision(t_S, traj_S)
            if E_rec is not None:
                results_S['recollision_energies'].append(E_rec)

    # Analyze results
    print("\n=== Results ===")
    print(f"R-enantiomer recollisions: {len(results_R['recollision_energies'])}")
    print(f"S-enantiomer recollisions: {len(results_S['recollision_energies'])}")

    if results_R['recollision_energies'] and results_S['recollision_energies']:
        CD = ChiralityAnalysis.circular_dichroism_signal(
            results_R['recollision_energies'], results_S['recollision_energies'])
        print(f"Circular Dichroism in recollision energy: {CD:.4f}")

    return {
        'R_enantiomer': results_R,
        'S_enantiomer': results_S,
        'properties_R': props_R,
        'properties_S': props_S,
        'laser': laser_rcp
    }

########################
# Visualization
########################

def plot_results(results):
    """Comprehensive visualization of results"""

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))

    # Plot 1: Sample trajectories
    ax = axes[0, 0]
    for i, (t, traj) in enumerate(results['R_enantiomer']['trajectories'][:10]):
        r = traj[:3, :].T
        ax.plot(r[:, 0], r[:, 1], 'b-', alpha=0.7, linewidth=1)
    for i, (t, traj) in enumerate(results['S_enantiomer']['trajectories'][:10]):
        r = traj[:3, :].T
        ax.plot(r[:, 0], r[:, 1], 'r-', alpha=0.7, linewidth=1)
    ax.set_xlabel('x (a.u.)')
    ax.set_ylabel('y (a.u.)')
    ax.set_title('Sample Trajectories\nBlue: R, Red: S')
    ax.grid(True, alpha=0.3)

    # Plot 2: Recollision energy distribution
    ax = axes[0, 1]
    energies_R = results['R_enantiomer']['recollision_energies']
    energies_S = results['S_enantiomer']['recollision_energies']

    if energies_R:
        ax.hist(energies_R, bins=30, alpha=0.7, color='blue', label='R-enantiomer')
    if energies_S:
        ax.hist(energies_S, bins=30, alpha=0.7, color='red', label='S-enantiomer')
    ax.set_xlabel('Recollision Energy (a.u.)')
    ax.set_ylabel('Counts')
    ax.set_title('Recollision Energy Distribution')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Plot 3: Molecular structures
    ax = axes[0, 2]
    symbols_R, coords_R, _ = build_chiral_molecule("fenchone", "R")
    symbols_S, coords_S, _ = build_chiral_molecule("fenchone", "S")

    coords_R_angstrom = coords_R * BOHR * 1e10
    coords_S_angstrom = coords_S * BOHR * 1e10

    ax.scatter(coords_R_angstrom[:, 0], coords_R_angstrom[:, 1],
               c='blue', s=50, label='R-enantiomer')
    ax.scatter(coords_S_angstrom[:, 0], coords_S_angstrom[:, 1],
               c='red', s=50, label='S-enantiomer', marker='^')
    ax.set_xlabel('x (Å)')
    ax.set_ylabel('y (Å)')
    ax.set_title('Molecular Structures')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Plot 4: Laser field
    ax = axes[1, 0]
    laser = results['laser']
    t_plot = np.linspace(-laser.tau_au, laser.tau_au, 1000)
    E_x = [laser.electric_field(t)[0] for t in t_plot]
    E_y = [laser.electric_field(t)[1] for t in t_plot]

    ax.plot(t_plot, E_x, 'b-', label='E_x')
    ax.plot(t_plot, E_y, 'r-', label='E_y')
    ax.set_xlabel('Time (a.u.)')
    ax.set_ylabel('Electric Field (a.u.)')
    ax.set_title('Circularly Polarized Laser Field')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Plot 5: Ionization rate
    ax = axes[1, 1]
    Ip = results['properties_R']['ionization_potential']
    E_fields = np.linspace(0.01, 0.1, 100)
    rates = [StrongFieldModels.ADK_rate(Ip, E * au_field) for E in E_fields]

    ax.semilogy(E_fields, rates, 'k-', linewidth=2)
    ax.set_xlabel('Electric Field (a.u.)')
    ax.set_ylabel('Ionization Rate (a.u.)')
    ax.set_title('ADK Tunnel Ionization Rate')
    ax.grid(True, alpha=0.3)

    # Plot 6: Chirality signal
    ax = axes[1, 2]
    if energies_R and energies_S:
        CD = ChiralityAnalysis.circular_dichroism_signal(energies_R, energies_S)
        enantiomers = ['R', 'S']
        mean_energies = [np.mean(energies_R), np.mean(energies_S)]
        std_energies = [np.std(energies_R), np.std(energies_S)]

        bars = ax.bar(enantiomers, mean_energies, yerr=std_energies,
                     capsize=5, color=['blue', 'red'], alpha=0.7)
        ax.set_ylabel('Mean Recollision Energy (a.u.)')
        ax.set_title(f'Chiral Signal\nCD = {CD:.4f}')
        ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('chiral_recollision_results.png', dpi=300, bbox_inches='tight')
    plt.show()

########################
# Main Execution
########################

if __name__ == "__main__":
    # Run the complete simulation
    results = run_chiral_simulation()

    # Generate comprehensive plots
    plot_results(results)

    print("\n=== Simulation Complete ===")
    print("Results saved to 'chiral_recollision_results.png'")

=== Strong-Field Chiral Recollision Simulation ===
Initializing molecular systems...
R-enantiomer: 25 atoms
S-enantiomer: 25 atoms
Calculating electronic structure...


/usr/local/lib/python3.12/dist-packages/pyscf/lib/linalg_helper.py:1405: UserWarning: /usr/local/lib/python3.12/dist-packages/pyscf/scf/addons.py:600: matrix a is not strictly positive definite
  warnings.warn('%s:%s: matrix a is not strictly positive definite' %
/usr/local/lib/python3.12/dist-packages/scipy/_lib/_util.py:1233: LinAlgWarning: Ill-conditioned matrix (rcond=8.64768e-35): result may not be accurate.
  return f(*arrays, *other_args, **kwargs)


RuntimeError: Ill geometry

In [4]:
pip install pyscf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 MB 13.8 MB/s eta 0:00:00
